In [15]:
deployment = "gpt4"
model = "gpt-4"
embedding_deployment = "embedding"

In [2]:
!pip install chromadb

In [3]:
! pip install unstructured

# 加载文档 （Document Loader）

In [3]:
from langchain.document_loaders import DirectoryLoader
loader = DirectoryLoader('./', glob="spotmax_intro.txt")
docs = loader.load()

In [5]:
!pip install pypdf

In [16]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("Spotmax_intro_cn_2020.pdf")
docs = loader.load()

# 切割文档
这个文本分割器是通用文本的推荐分割器。它由一个字符列表参数化。它会按顺序尝试在这些字符上进行分割，直到分块足够小。默认列表是["\n\n", "\n", " ", “"]。这样做的效果是尽可能地保持所有段落（然后是句子，然后是单词）在一起。

In [17]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 200,
    chunk_overlap = 10
)

In [18]:
splits = text_splitter.split_documents(docs)

In [19]:
print(splits[0])

page_content='SpotMax 打破成本约束，释放云端创造⼒ 构建⼤规模云端系统的挑战 云计算平台“按需获取”的特性使得⽤户可以随时获取所需计算资源，实现计算资源与业务规模的动态匹配。云计算时代计算资源已不再是构建⼤规模系统的瓶颈，多数情况下成本成为了企业在构建⼤规模应⽤时的最⼤挑战。 什么是 SpotMax?' metadata={'source': 'Spotmax_intro_cn_2020.pdf', 'page': 0}


In [7]:
persist_directory = 'data/'
!rm -rf ./data  

# 将数据存入向量数据库

In [20]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
# embedding = OpenAIEmbeddings() #如何直接调用OpenAI的 GPT 服务
embedding = OpenAIEmbeddings(deployment=embedding_deployment) #通过Azure的OpenAI GPT服务
vectordb = Chroma.from_documents(
    documents=splits,
    collection_name="spotmax",
    embedding=embedding,
    persist_directory=persist_directory
)
vectordb.persist()
print(vectordb._collection.count())

16


In [21]:
docs = vectordb.similarity_search("如何提高可用性",k=2)

In [22]:
docs[0]

Document(page_content='降低中断带来的服务器容量变化带来的服务能⼒影响，实现服务及数据的提前迁移。 4.MaxGroup将根据现货价格和终⽌率的变化持续优化集群结构。  5.⽀持智能的动态负载均衡，由于集群为多机型混合，不同机型的处理能⼒有差异，智能的动态负载均衡将实现按处理能⼒的负载分配，实现集群资源利⽤的最⼤化。', metadata={'page': 1, 'source': 'Spotmax_intro_cn_2020.pdf'})

# 利用向量数据库进行QA

https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/chains/retrieval_qa/prompt.py

In [26]:
from langchain.chains import RetrievalQA
from langchain.chat_models import AzureChatOpenAI
# from langchain.chat_models import ChatOpenAI #直接访问OpenAI的GPT服务

#llm = ChatOpenAI(model_name="gpt-4", temperature=0) #直接访问OpenAI的GPT服务
llm = AzureChatOpenAI(deployment_name = deployment, model_name=model, temperature=0.7, max_tokens=1000) #通过Azure的OpenAI服务
retriever=vectordb.as_retriever(
    search_type="mmr", search_kwargs={"k": 3}
    #search_type="similarity", search_kwargs={"k": 3}
)
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="refine", retriever=retriever,
                                 return_source_documents=False, verbose=True)



In [24]:
result = qa({"query": "什么是SpotMax?"})
print(result["result"])




> Entering new RetrievalQA chain...


Number of requested results 20 is greater than number of elements in index 16, updating n_results = 16



> Finished chain.
SpotMax是一个利用弹性计算资源进行云端系统成本优化的体系（包括相关产品及解决方案）。SpotMax旨在帮助用户使用云平台提供的弹性资源（如：AWS Spot Instance）在保证系统可用性，甚至获得更高可用性的前提下，实现成本的最大化节省。SpotMax来自于Mobvista降低成本的成功实践。


In [27]:
result = qa({"query": "SpotMax如何降低成本"})
print(result["result"])



> Entering new RetrievalQA chain...


Number of requested results 20 is greater than number of elements in index 16, updating n_results = 16



> Finished chain.
SpotMax通过使用云服务供应商的折扣实例（如AWS的Spot Instance）来降低成本。这些实例的价格通常远低于预留实例或按需实例，从而大大减少了云计算成本。SpotMax还具有弹性扩展和智能切换的功能，能够在折扣实例不可用时自动切换到其他类型的实例，无缝保障服务的连续性。使用SpotMax，用户能在保证稳定运行的同时，实现成本的有效降低。
